<a href="https://colab.research.google.com/github/AndryOut/Automatic-Timing-Sub-Fix/blob/main/Automatic_Timing_Sub_Fix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Fase 1
import subprocess
import sys

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--quiet", package])

install('pysrt')
install('pyass')
install('ipywidgets')

import pysrt
from google.colab import files
import ipywidgets as widgets
from IPython.display import display

def load_file():
    uploaded = files.upload()
    for filename in uploaded.keys():
        return filename

def read_ass_file(file_path):
    with open(file_path, 'r', encoding='utf-8-sig') as file:
        content = file.readlines()
    return content

def find_top_alignments(styles, dialogue):
    top_alignments = ['7', '8', '9']
    top_styles = []
    alignment_index = None
    for line in styles:
        if line.startswith('Format:'):
            format_parts = [x.strip() for x in line.split(':')[1].split(',')]
            if 'Alignment' in format_parts:
                alignment_index = format_parts.index('Alignment')
        elif line.startswith('Style:') and alignment_index is not None:
            style_parts = line.split(',')
            alignment = style_parts[alignment_index].strip()
            if alignment in top_alignments:
                style_name = style_parts[0].replace('Style:', '').strip()
                top_styles.append(style_name)

    if not top_styles or not any(line.split(',')[3].strip() in top_styles for line in dialogue):
        top_styles = find_alignments_in_dialogue(dialogue, top_alignments)

    return top_styles

def find_alignments_in_dialogue(dialogue, top_alignments):
    dialogue_with_alignments = []
    for line in dialogue:
        if any(alignment in line for alignment in top_alignments) or '{\\an8}' in line:
            dialogue_with_alignments.append(line)
    return dialogue_with_alignments

def filter_dialogue_by_style(dialogue, top_styles):
    filtered_dialogue = [line for line in dialogue if line.split(',')[3].strip() in top_styles or '{\\an8}' in line]
    discarded_dialogue = [line for line in dialogue if line.split(',')[3].strip() not in top_styles and '{\\an8}' not in line]
    return filtered_dialogue, discarded_dialogue

def write_ass_file(header, dialogue, format_line, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        file.writelines(header + [format_line] + dialogue)

def write_srt_file(discarded_dialogue, file_path):
    subs = pysrt.SubRipFile()
    for idx, line in enumerate(discarded_dialogue):
        parts = line.split(',')
        start = parts[1].replace('.', ',')
        end = parts[2].replace('.', ',')
        text = ','.join(parts[9:]).strip()
        sub = pysrt.SubRipItem(index=idx+1, start=start, end=end, text=text)
        subs.append(sub)
    subs.save(file_path, encoding='utf-8')

def process_ass_file(file_path):
    lines = read_ass_file(file_path)
    header_section = []
    dialogue_section = []
    capture_dialogue = False
    capture_styles = False
    styles_section = []
    format_line_events = ""
    format_line_styles = ""
    for line in lines:
        if line.startswith('[V4+ Styles]'):
            capture_styles = True
            header_section.append(line)
            continue
        if line.startswith('[Events]'):
            capture_styles = False
            capture_dialogue = True
            header_section.append(line)
            continue
        if capture_dialogue and line.startswith('Dialogue:'):
            dialogue_section.append(line)
        elif not capture_dialogue:
            header_section.append(line)
        if capture_styles and line.startswith('Format:'):
            format_line_styles = line
        if not capture_styles and line.startswith('Format:'):
            format_line_events = line
        if capture_styles:
            styles_section.append(line)

    top_styles = find_top_alignments(styles_section, dialogue_section)
    filtered_dialogue, discarded_dialogue = filter_dialogue_by_style(dialogue_section, top_styles)

    return header_section, filtered_dialogue, discarded_dialogue, format_line_events

def process_srt_file(file_path):
    subs = pysrt.open(file_path, encoding='utf-8')
    an8_subs = pysrt.SubRipFile()
    discarded_subs = pysrt.SubRipFile()

    for sub in subs:
        if '{\\an8}' in sub.text:
            an8_subs.append(sub)
        else:
            discarded_subs.append(sub)

    an8_subs.save('On Top.srt', encoding='utf-8')
    discarded_subs.save('Sub.srt', encoding='utf-8')

    print('Sottotitoli processati e salvati in output.srt')
    print('Righe scartate salvate in Sub.srt')

def on_button_yes_clicked(b):
    print("Per una precisione maggiore e per evitare conflitti nei prossimi script, dovrai prima togliere i cartelli dai sub.")
    print("Una volta tolti, riavvia lo script.")

def on_button_no_clicked(b):
    input_file_path = load_file()

    if input_file_path.endswith('.srt'):
        process_srt_file(input_file_path)
    else:
        output_ass_file_path = 'On Top.ass'
        output_srt_file_path = 'Sub.srt'
        header, dialogue, discarded_dialogue, format_line = process_ass_file(input_file_path)
        write_ass_file(header, dialogue, format_line, output_ass_file_path)
        write_srt_file(discarded_dialogue, output_srt_file_path)
        print(f'Sottotitoli processati e salvati in {output_ass_file_path}')
        print(f'Righe scartate salvate in {output_srt_file_path}')

def ask_about_signs():
    button_yes = widgets.Button(description="Sì")
    button_no = widgets.Button(description="No")
    button_yes.on_click(on_button_yes_clicked)
    button_no.on_click(on_button_no_clicked)
    display(widgets.HBox([widgets.Label("I sub che stai caricando hanno i cartelli?"), button_yes, button_no]))

def main():
    ask_about_signs()

if __name__ == '__main__':
    main()

In [ ]:
# @title Fase 2
!pip install pysrt pydub librosa

import pysrt
from pydub import AudioSegment
import librosa
import numpy as np
from google.colab import files

# Funzione per convertire i millisecondi in SubRipTime
def milliseconds_to_subrip_time(milliseconds):
    hours = int(milliseconds // 3600000)
    minutes = int((milliseconds % 3600000) // 60000)
    seconds = int((milliseconds % 60000) // 1000)
    milliseconds = int(milliseconds % 1000)
    return pysrt.SubRipTime(hours=hours, minutes=minutes, seconds=seconds, milliseconds=milliseconds)

# Funzione per rilevare i segmenti audio con Pydub e Librosa
def get_audio_segments(audio_file, silence_threshold=320):
    # Carica l'audio con Pydub
    audio = AudioSegment.from_file(audio_file)
    audio.export("temp.wav", format="wav")

    # Carica l'audio con Librosa
    y, sr = librosa.load("temp.wav", sr=None)

    # Rileva i segmenti non silenziosi
    intervals = librosa.effects.split(y, top_db=20)

    # Converte i segmenti in millisecondi
    segments = []
    for start, end in intervals:
        segments.append((start / sr * 1000, end / sr * 1000))

    return segments

# Funzione per rilevare i picchi audio per aggiustare i time stamps
def adjust_timestamps_based_on_peaks(subs, audio_file):
    audio_segments = get_audio_segments(audio_file)
    for sub in subs:
        start_ms = sub.start.ordinal
        end_ms = sub.end.ordinal

        # Trova il primo picco audio dopo l'inizio del time stamp
        for segment_start, segment_end in audio_segments:
            if segment_start >= start_ms:
                # Se il primo picco audio rilevato supera il range massimo di 0,300 secondi, cerca un altro picco più vicino
                if segment_start - start_ms > 550:
                    found = False
                    for closer_segment_start, closer_segment_end in audio_segments:
                        if start_ms <= closer_segment_start <= start_ms + 500:
                            sub.start = milliseconds_to_subrip_time(closer_segment_start)
                            found = True
                            break
                    if not found:
                        sub.start = milliseconds_to_subrip_time(start_ms)
                else:
                    sub.start = milliseconds_to_subrip_time(segment_start)
                break

        # Se il time stamp finale della riga attuale è già su un picco audio, lo cerca dopo il time stamp finale
        is_on_peak = False
        for segment_start, segment_end in audio_segments:
            if segment_start <= end_ms <= segment_end:
                is_on_peak = True
                break

        if is_on_peak:
            # Cerca un picco audio dopo il time stamp finale nel range di 0,150 secondi
            found = False
            for segment_start, segment_end in audio_segments:
                if end_ms <= segment_start <= end_ms + 150:
                    sub.end = milliseconds_to_subrip_time(segment_start)
                    found = True
                    break
            if not found:
                sub.end = milliseconds_to_subrip_time(end_ms)

        # Trova l'ultimo picco audio prima della fine del time stamp
        max_ranges = [300, 400]
        for max_range in max_ranges:
            found = False
            for segment_start, segment_end in reversed(audio_segments):
                if segment_end <= end_ms and segment_start >= start_ms:
                    if end_ms - segment_end <= max_range:
                        sub.end = milliseconds_to_subrip_time(segment_end)
                        found = True
                        break
            if found:
                break

        # Se il picco audio rilevato è troppo lontano, lo cerca di nuovo
        if end_ms - sub.end.ordinal > 410:
            for segment_start, segment_end in audio_segments:
                if segment_end <= end_ms and segment_start >= start_ms:
                    if end_ms - segment_end <= 300:
                        sub.end = milliseconds_to_subrip_time(segment_end)
                        break
            else:
                sub.end = milliseconds_to_subrip_time(end_ms)

    return subs

# Funzione per aggiungere lead-in e lead-out ai segmenti
def add_lead_in_out(segments, original_subs, lead_in=200, lead_out=500):
    adjusted_segments = []
    for i, (start, end) in enumerate(segments):
        original_start_ms = original_subs[i].start.ordinal
        if start == original_start_ms:
            new_start = start  # Non aggiungere lead-in se è uguale al time stamp originale
        else:
            new_start = max(0, start - lead_in)  # Aggiungi lead-in altrimenti
        new_end = end + lead_out  # Aggiungi lead-out a tutto
        adjusted_segments.append((new_start, new_end))
    return adjusted_segments

# Funzione per collegare segmenti senza overlap con spazio di 0,000 secondi
def adjust_segments_for_overlap(segments, max_lead_out=800, lead_in=200, max_lead_in=270, lead_out=500):
    adjusted_segments = []
    for i in range(len(segments) - 1):
        start, end = segments[i]
        next_start, next_end = segments[i + 1]

        if (next_start - end) <= max_lead_out:
            if (next_start - end) > lead_out:
                remaining_time = next_start - end - lead_out
                end = next_start - remaining_time  # collega con spazio di 0,000 secondi
            else:
                end = next_start  # collega con spazio di 0,000 secondi
        else:
            if (next_start - end) > lead_in and (next_start - end) < max_lead_in:
                end = next_start - lead_in

        adjusted_segments.append((start, end))

    adjusted_segments.append(segments[-1])
    return adjusted_segments

# Chiede di caricare prima un file audio e dopo un file SRT
uploaded_audio = files.upload()
audio_file = list(uploaded_audio.keys())[0]

uploaded_srt = files.upload()
srt_file = list(uploaded_srt.keys())[0]

# Carica il file SRT originale
subs = pysrt.open(srt_file, encoding='utf-8')
original_subs = pysrt.open(srt_file, encoding='utf-8')  # Fai una copia del file SRT originale

# Aggiusta i time stamps delle righe in base ai picchi audio rilevati
subs = adjust_timestamps_based_on_peaks(subs, audio_file)

# Ottieni i segmenti audio con Pydub e Librosa
audio_segments = get_audio_segments(audio_file)

# Non unisce i segmenti, mantiene solo quelli esistenti
final_segments = [(sub.start.ordinal, sub.end.ordinal) for sub in subs]

# Aggiunge lead-in e lead-out ai segmenti
adjusted_segments = add_lead_in_out(final_segments, original_subs)

# Regola i segmenti per evitare overlap e collegarli con spazio di 0,000 secondi
adjusted_segments = adjust_segments_for_overlap(adjusted_segments)

# Crea il file SRT basato sui segmenti finali e mantiene il testo originale
for sub, (start, end) in zip(subs, adjusted_segments):
    sub.start = milliseconds_to_subrip_time(start)
    sub.end = milliseconds_to_subrip_time(end)

# Salva il file SRT finale
output_file = 'adjusted_' + srt_file
subs.save(output_file, encoding='utf-8')

print(f"File SRT salvato come {output_file}")

In [ ]:
# @title Fase 3
!pip install scenedetect[opencv] pysrt

from scenedetect import open_video, SceneManager, FrameTimecode
from scenedetect.detectors import AdaptiveDetector, ContentDetector
import pysrt

# Funzione per esportare i risultati in formato SRT con precisione al millisecondo
def export_srt(scene_list, output_path='scene_timestamps.srt'):
    def seconds_to_timecode(seconds):
        hrs = int(seconds // 3600)
        mins = int((seconds % 3600) // 60)
        secs = int(seconds % 60)
        millis = int((seconds % 1) * 1000)
        return f"{hrs:02}:{mins:02}:{secs:02},{millis:03}"

    with open(output_path, 'w') as f:
        for i, scene in enumerate(scene_list):
            start_time = scene[0].get_seconds()
            end_time = scene[1].get_seconds() - 0.001  # Sottrai 1 millisecondo per evitare sovrapposizioni

            start_timecode = seconds_to_timecode(start_time)
            end_timecode = seconds_to_timecode(end_time)

            f.write(f"{i+1}\n")
            f.write(f"{start_timecode} --> {end_timecode}\n")
            f.write(f"Scene {i+1}\n\n")

# Funzione per calcolare la discrepanza costante
def calculate_discrepancy(scene_list, srt_path):
    # Carica il file SRT esistente
    subs = pysrt.open(srt_path, encoding='utf-8')

    # Calcola la discrepanza tra i cambi scena e i timecode dei sottotitoli
    discrepancies = []
    count = min(len(scene_list), len(subs))
    for i in range(count):
        scene_start = scene_list[i][0].get_seconds()
        subtitle_start = subs[i].start.ordinal / 1000
        discrepancy = scene_start - subtitle_start
        discrepancies.append(discrepancy)

    # Ritorna la discrepanza media
    return sum(discrepancies) / len(discrepancies)

# Funzione per trovare l'offset più vicino ai valori predefiniti
def find_closest_offset(discrepancy, possible_offsets):
    closest_offset = min(possible_offsets, key=lambda x: abs(x - discrepancy))
    return closest_offset

# Funzione per applicare un offset globale al file SRT
def apply_global_offset_to_srt(input_path, output_path, offset):
    def apply_offset(timecode, offset):
        timecode.ordinal += int(offset * 1000)  # Converte i secondi in millisecondi
        return timecode

    # Carica il file SRT esistente
    subs = pysrt.open(input_path, encoding='utf-8')

    # Applica l'offset ai timecode di inizio e fine
    for sub in subs:
        sub.start = apply_offset(sub.start, offset)
        sub.end = apply_offset(sub.end, offset)

    # Salva il file SRT con i timecode modificati
    subs.save(output_path, encoding='utf-8')

# Caricamento del Video
video_path = '/content/Ep.mkv'

# Verifica se il file video è stato caricato correttamente
if not video_path:
    raise FileNotFoundError("Il file video non è stato trovato.")

# Caricamento del Video
video_manager = open_video(video_path)

# Crea un SceneManager con AdaptiveDetector e ContentDetector
scene_manager = SceneManager()
adaptive_detector = AdaptiveDetector(adaptive_threshold=19)
content_detector = ContentDetector(threshold=19)
scene_manager.add_detector(adaptive_detector)
scene_manager.add_detector(content_detector)

# Rileva le scene per tutta la durata del video
scene_manager.detect_scenes(video_manager)
scene_list = scene_manager.get_scene_list()

# Defininisce il percorso del file SRT di output
srt_output_path = 'scene_timestamps.srt'

# Esportare i risultati in formato SRT
export_srt(scene_list, output_path=srt_output_path)

# Calcola la discrepanza costante in base ai cambi scena e ai sottotitoli
discrepancy = calculate_discrepancy(scene_list, srt_output_path)

# Definisce i possibili offset negativi in secondi
possible_offsets = [-0.011, -0.021, -0.031, -0.041]

# Trova l'offset più vicino
best_offset = find_closest_offset(discrepancy, possible_offsets)

# Applica l'offset globale al file SRT
apply_global_offset_to_srt(srt_output_path, 'scene_timestamps_adjusted.srt', best_offset)

print(f"Scene rilevate: {len(scene_list)}")
for i, scene in enumerate(scene_list):
    print(f'Scena {i+1}: Inizio: {scene[0].get_timecode()}, Fine: {scene[1].get_timecode()}')

print(f'File SRT con offset globale applicato creato con successo: scene_timestamps_adjusted.srt')
print(f'Offset applicato: {best_offset:.3f} secondi')

In [ ]:
# @title Fase 4
!pip install pysrt pydub librosa

# Importa le librerie
import pysrt
from pydub import AudioSegment
import librosa
import numpy as np

# Funzione per convertire i millisecondi in SubRip Time
def milliseconds_to_subrip_time(milliseconds):
    hours = int(milliseconds // 3600000)
    minutes = int((milliseconds % 3600000) // 60000)
    seconds = int((milliseconds % 60000) // 1000)
    milliseconds = int(milliseconds % 1000)
    return pysrt.SubRipTime(hours=hours, minutes=minutes, seconds=seconds, milliseconds=milliseconds)

# Funzione per rilevare i picchi audio
def get_audio_peaks(audio_file):
    y, sr = librosa.load(audio_file)
    onset_env = librosa.onset.onset_strength(y=y, sr=sr)
    peaks = librosa.onset.onset_detect(onset_envelope=onset_env, sr=sr, backtrack=True)
    peak_times = librosa.frames_to_time(peaks, sr=sr)
    return peak_times

# Funzione per rilevare cambi scena prima del time stamps iniziali della riga
def adjust_subs_based_on_scenes(original_subs, scene_subs):
    adjusted_subs = original_subs
    for idx, sub in enumerate(adjusted_subs):
        start_replaced = False  # Controlla se il timestamp iniziale è stato sostituito
        for scene in reversed(scene_subs):
            scene_end = scene.end.ordinal
            sub_start = sub.start.ordinal
            # Controlla se il timestamp iniziale della riga è vicino al cambio scena precedente
            if 0 < (sub_start - scene_end) <= 250:
                sub.start = milliseconds_to_subrip_time(scene_end)
                start_replaced = True  # Imposta la variabile a True se il timestamp iniziale è stato sostituito
                break
        # Sotto-regola: se c'è una riga precedente
        if idx > 0:
            prev_sub = adjusted_subs[idx - 1]
            prev_sub_end = prev_sub.end.ordinal
            prev_sub_start = prev_sub.start.ordinal
            if prev_sub_end > scene_end and scene_end >= prev_sub_start:
                if start_replaced:
                    prev_sub.end = milliseconds_to_subrip_time(scene_end - 0)
            else:
                # Sotto-regola: se la riga precedente non è sopra al cambio scena
                if scene_end >= prev_sub_end:
                    if start_replaced:
                        sub.start = milliseconds_to_subrip_time(scene_end)
                # Controllo: se non è stato sostituito il timestamp iniziale della riga attuale, non sostituisce il timestamp finale della riga precedente
                if not start_replaced:
                    prev_sub.end = milliseconds_to_subrip_time(prev_sub_end)
    return adjusted_subs

# Funzione per rilevare e sostituire il timestamp iniziale della riga se il cambio scena è entro 0,250 secondi
def adjust_sub_start_based_on_scene_change(original_subs, scene_subs):
    for sub in original_subs:
        sub_start = sub.start.ordinal
        for scene in scene_subs:
            scene_start = scene.start.ordinal
            # Controlla se il cambio scena si trova entro 0,250 secondi dopo il timestamp iniziale della riga attuale
            if 0 < (scene_start - sub_start) <= 250:
                sub.start = milliseconds_to_subrip_time(scene_start)
                break
    return original_subs

# Funzione per aggiungere lead-in ai timestamp iniziali troppo vicini ai picchi audio
def add_lead_in_to_peaks(subs, audio_peaks):
    min_lead_in = 50  # Minimo lead-in in millisecondi
    max_lead_in = 200  # Massimo lead-in in millisecondi
    additional_lead_in = 200  # Lead-in aggiuntivo in millisecondi
    gap_threshold = 300  # Soglia di distacco in millisecondi

    for idx, sub in enumerate(subs):
        sub_start = sub.start.ordinal
        for peak in audio_peaks:
            peak_time = int(peak * 1000)  # Converte i picchi in millisecondi
            lead_in_duration = peak_time - sub_start
            if 0 < lead_in_duration < min_lead_in:
                # Aggiunge lead-in aggiuntivo se lead-in è troppo breve
                sub.start = milliseconds_to_subrip_time(sub_start - additional_lead_in)
                break
            elif min_lead_in <= lead_in_duration <= max_lead_in:
                # Mantiene il timestamp inalterato se il lead-in è sufficiente
                break
        # Sotto-regola: evita l'overlap con la riga precedente
        if idx > 0:
            prev_sub_end = subs[idx - 1].end.ordinal
            if sub.start.ordinal < prev_sub_end:
                sub.start = milliseconds_to_subrip_time(prev_sub_end + 0)

        # Sotto-regola: attacca il time stamps finale della riga attuale al time stamps iniziale della riga successiva se c'è un range di 0,300 secondi
        if idx < len(subs) - 1:
            next_sub_start = subs[idx + 1].start.ordinal
            if 0 < (next_sub_start - sub.end.ordinal) <= gap_threshold:
                sub.end = milliseconds_to_subrip_time(next_sub_start)

    return subs

# Funzione per sostituire il timestamp finale della riga con il timestamp iniziale del cambio scena successivo
def adjust_sub_end_based_on_next_scene_change(original_subs, scene_subs):
    max_range = 400  # Massimo range in millisecondi

    for idx, sub in enumerate(original_subs):
        sub_end = sub.end.ordinal
        for scene in scene_subs:
            scene_start = scene.start.ordinal
            # Controlla se il timestamp finale della riga è entro 0,800 secondi dal timestamp iniziale del cambio scena successivo
            if 0 < (scene_start - sub_end) <= max_range:
                # Sotto-regola: verifica se c'è un'altra riga tra la riga attuale e il cambio scena
                if idx < len(original_subs) - 1 and original_subs[idx + 1].start.ordinal < scene_start:
                    break
                sub.end = milliseconds_to_subrip_time(scene_start)
                break  # Esce dal loop dopo aver trovato il cambio scena entro il range
    return original_subs

# Funzione per sostituire il timestamp finale della riga con il timestamp finale del cambio scena precedente
def adjust_sub_end_based_on_previous_scene_change(original_subs, scene_subs, audio_peaks):
    max_range = 700  # Massimo range in millisecondi
    extended_range = 850  # Range esteso se non ci sono picchi audio

    for sub in original_subs:
        sub_end = sub.end.ordinal
        for scene in reversed(scene_subs):
            scene_end = scene.end.ordinal
            range_to_check = max_range

            # Controlla se ci sono picchi audio tra sub_end e scene_end
            peaks_in_range = any(peak for peak in audio_peaks if sub_end < peak * 1000 < scene_end)

            # Se non ci sono picchi audio, estende il range massimo a 850 millisecondi
            if not peaks_in_range:
                range_to_check = extended_range

            # Controlla se il timestamp finale della riga è entro il range determinato
            if 0 < (sub_end - scene_end) <= range_to_check:
                sub.end = milliseconds_to_subrip_time(scene_end)
                break  # Esce dal loop dopo aver trovato il cambio scena entro il range
    return original_subs

# Carica i file necessari
original_subs = pysrt.open('adjusted_Sub.srt', encoding='utf-8')
scene_subs = pysrt.open('scene_timestamps_adjusted.srt', encoding='utf-8')
audio_peaks = get_audio_peaks('Vocali.wav')

# Funzione per aggiungere lead-in ai timestamp iniziali troppo vicini ai picchi audio
adjusted_subs = add_lead_in_to_peaks(original_subs, audio_peaks)

# Funzione per rilevare cambi scena prima del time stamps iniziali della riga
adjusted_subs = adjust_subs_based_on_scenes(original_subs, scene_subs)

# Funzione per rilevare e sostituire il timestamp iniziale della riga se il cambio scena è entro 0,250 secondi
adjusted_subs = adjust_sub_start_based_on_scene_change(original_subs, scene_subs)

# Funzione per sostituire il timestamp finale della riga con il timestamp iniziale del cambio scena successivo
adjusted_subs = adjust_sub_end_based_on_next_scene_change(original_subs, scene_subs)
adjusted_subs = adjust_sub_end_based_on_previous_scene_change(adjusted_subs, scene_subs, audio_peaks)

# Salva il nuovo file SRT
adjusted_subs.save('Final.srt', encoding='utf-8')
print("Script completato e sottotitoli aggiornati salvati come 'adjusted_subs.srt'")